## Combbine all diagnosis

This method take diagnosis from images, clinical, and diagnosis sheet, and creates one ground truth (where all three agree) and one majority vote (where two agree) diagnosis files. 

In [108]:
import pandas as pd
import math
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
#this file is the metadata file that one can get from downloading MRI images from ADNI
# img = pd.read_csv("../preprocess_images/MRI_meta2.csv")
img = pd.read_csv("ADNI12go.csv")
comb = pd.read_csv("DXSUM_PDXCONV_ADNIALL.csv")[["RID", "PTID" , "PHASE"]].rename(columns={"PHASE":"Phase"})

In [109]:
clinical['PHC_Diagnosis'].unique()

array([ 1.,  2.,  3., nan])

In [110]:
def read_diagnose(file_path: str = 'DXSUM_PDXCONV_ADNIALL.csv', verbose=False):
    # Read diagnostic summary
    diagnostic_summary = pd.read_csv(file_path, index_col='PTID').rename(columns={"PHASE":"Phase"})
    diagnostic_summary = diagnostic_summary.sort_values(by=["update_stamp"], ascending=True)
    # Create dictionary
    diagnostic_dict: dict = {}
    for key, data in diagnostic_summary.iterrows():
        # Iterate for each row of the document
        phase: str = data['Phase']
        diagnosis: float = -1.
        if phase == "ADNI1":
            diagnosis = data['DIAGNOSIS']
        elif phase == "ADNI2" or phase == "ADNIGO":
            diagnosis = data['DIAGNOSIS']
        elif phase == "ADNI3":
            diagnosis = data['DIAGNOSIS']
        else:
            print(f"ERROR: Not recognized study phase {phase}")
            exit(1)
        # Update dictionary
        if not math.isnan(diagnosis):
            diagnostic_dict[key] = diagnosis
    if verbose:
        print_diagnostic_dict_summary(diagnostic_dict)
    return diagnostic_dict


def print_diagnostic_dict_summary(diagnostic_dict: dict):
    print(f"Number of diagnosed patients: {len(diagnostic_dict.items())}\n")
    n_NL = 0
    n_MCI = 0
    n_AD = 0
    for (key, data) in diagnostic_dict.items():
        if data == 1:
            n_NL += 1
        if data == 2:
            n_MCI += 1
        if data == 3:
            n_AD += 1
    print(f"Number of NL patients: {n_NL}\n"
          f"Number of MCI patients: {n_MCI}\n"
          f"Number of AD patients: {n_AD}\n")

In [111]:
d = read_diagnose()
print_diagnostic_dict_summary(d)

ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4
ERROR: Not recognized study phase ADNI4


In [112]:
new = pd.DataFrame.from_dict(d, orient='index').reset_index()

In [113]:
new[0].unique()


array([ 2.,  3.,  1., -1.])

In [114]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,PHC_Ethnicity,...,PHC_MEM_PreciseFilter,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,...,1,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,...,1,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,...,1,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,...,1,0.066,0.313,1.0,0.620,0.342,1.0,0.264,0.547,1.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,...,1,0.002,0.336,1.0,0.460,0.374,1.0,0.264,0.547,1.0


In [115]:
clinical["year"] = clinical["EXAMDATE"].str[:4]

In [116]:
clinical["Subject"] = clinical["SUBJID"].str.replace("ADNI_", "").str.replace("s", "S")

In [117]:
len(clinical["Subject"].unique())

1565

In [118]:
c = comb.merge(clinical, on = ["RID", "Phase"])

In [119]:
c = c.drop("Subject", axis =1)

In [120]:
c = c.rename(columns = {"PTID":"Subject"})

In [121]:
c

,RID,Subject,Phase,SUBJID,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,year
0,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
1,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
2,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,3,1.0,16.0,...,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0,2008
3,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
4,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54128,7123,035_S_7123,ADNI3,NaN,sc,sc,2023-02-28,1,2.0,14.0,...,0.503,0.746,0.0,0.071,0.645,0.0,-0.228,0.780,0.0,2023
54129,2249,052_S_2249,ADNI3,ADNI_052_S_2249,init,m90,2018-08-27,8,1.0,15.0,...,0.487,0.640,0.0,-0.054,0.435,1.0,0.891,0.693,0.0,2018
54130,6949,016_S_6949,ADNI3,NaN,sc,sc,2021-05-17,1,2.0,14.0,...,0.503,0.746,0.0,0.064,0.636,0.0,0.547,0.801,0.0,2021
54131,6338,177_S_6338,ADNI3,NaN,sc,sc,2018-05-16,1,2.0,20.0,...,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0,2018


In [84]:
len(c['Subject'].unique()),len(m['Subject'].unique())

(2943, 3249)

In [122]:
img["year"] = img["Acq Date"].str[5:].str.replace("/", "")

In [123]:
img = img.replace(["CN", "MCI", "AD"], [ 0, 1, 2])

In [124]:
c

,RID,Subject,Phase,SUBJID,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,year
0,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
1,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
2,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,3,1.0,16.0,...,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0,2008
3,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
4,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54128,7123,035_S_7123,ADNI3,NaN,sc,sc,2023-02-28,1,2.0,14.0,...,0.503,0.746,0.0,0.071,0.645,0.0,-0.228,0.780,0.0,2023
54129,2249,052_S_2249,ADNI3,ADNI_052_S_2249,init,m90,2018-08-27,8,1.0,15.0,...,0.487,0.640,0.0,-0.054,0.435,1.0,0.891,0.693,0.0,2018
54130,6949,016_S_6949,ADNI3,NaN,sc,sc,2021-05-17,1,2.0,14.0,...,0.503,0.746,0.0,0.064,0.636,0.0,0.547,0.801,0.0,2021
54131,6338,177_S_6338,ADNI3,NaN,sc,sc,2018-05-16,1,2.0,20.0,...,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0,2018


In [125]:
c["PHC_Diagnosis"] = c["PHC_Diagnosis"] -1

In [126]:
c["PHC_Diagnosis"].unique()

array([ 0.,  2.,  1., nan])

In [127]:
new

,index,0
0,011_S_0002,2.0
1,011_S_0003,3.0
2,011_S_0005,1.0
3,011_S_0008,2.0
4,022_S_0007,3.0
...,...,...
3227,114_S_10321,-1.0
3228,082_S_10219,-1.0
3229,033_S_10326,-1.0
3230,011_S_10294,-1.0


In [128]:
new[0] = new[0].astype(int) -1

In [129]:
new = new.rename(columns = {"index":"Subject", 0:"GroupN"})

In [130]:
new

,Subject,GroupN
0,011_S_0002,1
1,011_S_0003,2
2,011_S_0005,0
3,011_S_0008,1
4,022_S_0007,2
...,...,...
3227,114_S_10321,-2
3228,082_S_10219,-2
3229,033_S_10326,-2
3230,011_S_10294,-2


In [131]:
c

,RID,Subject,Phase,SUBJID,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,year
0,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
1,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
2,2,011_S_0002,ADNI1,ADNI_011_S_0002,m36,m36,2008-08-27,3,1.0,16.0,...,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0,2008
3,2,011_S_0002,ADNI1,ADNI_011_S_0002,bl,bl,2005-09-08,1,1.0,16.0,...,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0,2005
4,2,011_S_0002,ADNI1,ADNI_011_S_0002,m06,m06,2006-03-06,2,1.0,16.0,...,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54128,7123,035_S_7123,ADNI3,NaN,sc,sc,2023-02-28,1,2.0,14.0,...,0.503,0.746,0.0,0.071,0.645,0.0,-0.228,0.780,0.0,2023
54129,2249,052_S_2249,ADNI3,ADNI_052_S_2249,init,m90,2018-08-27,8,1.0,15.0,...,0.487,0.640,0.0,-0.054,0.435,1.0,0.891,0.693,0.0,2018
54130,6949,016_S_6949,ADNI3,NaN,sc,sc,2021-05-17,1,2.0,14.0,...,0.503,0.746,0.0,0.064,0.636,0.0,0.547,0.801,0.0,2021
54131,6338,177_S_6338,ADNI3,NaN,sc,sc,2018-05-16,1,2.0,20.0,...,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0,2018


In [23]:
m = new.merge(c, on = "Subject", how = "outer").merge(img, on = "Subject", how = "outer")

In [24]:
m.columns

Index(['Subject', 'GroupN', 'RID', 'Phase', 'SUBJID', 'VISCODE', 'VISCODE2',
       'EXAMDATE', 'PHC_Visit', 'PHC_Sex', 'PHC_Education', 'PHC_Ethnicity',
       'PHC_Race', 'PHC_Age_Cognition', 'PHC_Diagnosis', 'PHC_MEM',
       'PHC_MEM_SE', 'PHC_MEM_PreciseFilter', 'PHC_EXF', 'PHC_EXF_SE',
       'PHC_EXF_PreciseFilter', 'PHC_LAN', 'PHC_LAN_SE',
       'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'year_x', 'Image Data ID', 'Group', 'Sex',
       'Age', 'Visit', 'Modality', 'Description', 'Type', 'Acq Date', 'Format',
       'Downloaded', 'year_y'],
      dtype='object')

In [25]:
m[["GroupN", "PHC_Diagnosis", "Group"]]

,GroupN,PHC_Diagnosis,Group
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
55147,NaN,NaN,1.0
55148,NaN,NaN,1.0
55149,NaN,NaN,2.0
55150,NaN,NaN,2.0


In [26]:
m = m[["Subject", "GroupN", "Group", "PHC_Diagnosis", "Phase"]].drop_duplicates()

In [27]:
m = m.dropna(subset = ["GroupN", "Group", "PHC_Diagnosis"], how="all").drop_duplicates()
m

,Subject,GroupN,Group,PHC_Diagnosis,Phase
0,011_S_0002,1.0,0.0,0.0,ADNI1
9,011_S_0002,1.0,0.0,0.0,ADNIGO
10,011_S_0002,1.0,0.0,0.0,ADNI2
11,011_S_0002,1.0,0.0,1.0,ADNI2
35,011_S_0002,1.0,0.0,1.0,ADNI3
...,...,...,...,...,...
55147,094_S_0964,NaN,1.0,NaN,NaN
55148,062_S_1091,NaN,1.0,NaN,NaN
55149,014_S_0357,NaN,2.0,NaN,NaN
55150,073_S_1207,NaN,2.0,NaN,NaN


In [28]:
m.loc[m["PHC_Diagnosis"].isna() & m["Group"].isna(), "Group"] = m.loc[m["PHC_Diagnosis"].isna() & m["Group"].isna(), "GroupN"]
m.loc[m["PHC_Diagnosis"].isna() & m["Group"].isna(), "DX"] = m.loc[m["PHC_Diagnosis"].isna() & m["Group"].isna(), "GroupN"]

In [29]:
m1 = m[m["GroupN"] == m["Group"]]
m3 = m[m["GroupN"] == m["PHC_Diagnosis"]]
m4 = m[m["Group"] == m["PHC_Diagnosis"]]
m2 = m1[m1["Group"] == m1["PHC_Diagnosis"]]

In [30]:
m1 = m1[["Subject", "GroupN", "Group", "PHC_Diagnosis", "Phase"]]
m1

,Subject,GroupN,Group,PHC_Diagnosis,Phase
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
119,022_S_0007,2.0,2.0,2.0,ADNI1
197,023_S_0031,0.0,0.0,0.0,ADNI1
233,023_S_0031,0.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
55126,114_S_10321,-2.0,-2.0,NaN,NaN
55127,082_S_10219,-2.0,-2.0,NaN,NaN
55128,033_S_10326,-2.0,-2.0,NaN,NaN
55129,011_S_10294,-2.0,-2.0,NaN,NaN


In [85]:
len(m1['Subject'].unique())

738

In [31]:
m1.loc[m1["PHC_Diagnosis"].isna(), "PHC_Diagnosis"] = m1.loc[m1["PHC_Diagnosis"].isna(), "Group"]

In [32]:
m3 = m3[["Subject", "GroupN", "Group", "PHC_Diagnosis", "Phase"]]
m3

,Subject,GroupN,Group,PHC_Diagnosis,Phase
11,011_S_0002,1.0,0.0,1.0,ADNI2
35,011_S_0002,1.0,0.0,1.0,ADNI3
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
106,011_S_0008,1.0,0.0,1.0,ADNI2
...,...,...,...,...,...
54839,013_S_7118,0.0,NaN,0.0,ADNI3
54840,153_S_7113,0.0,NaN,0.0,ADNI3
54841,035_S_7120,2.0,NaN,2.0,ADNI3
54844,082_S_7117,1.0,NaN,1.0,ADNI3


In [33]:
m3.loc[m3["Group"].isna(), "Group"] = m3.loc[m3["Group"].isna(), "GroupN"]

In [34]:
m4 = m4[["Subject", "GroupN", "Group", "PHC_Diagnosis", "Phase"]]
m4

,Subject,GroupN,Group,PHC_Diagnosis,Phase
0,011_S_0002,1.0,0.0,0.0,ADNI1
9,011_S_0002,1.0,0.0,0.0,ADNIGO
10,011_S_0002,1.0,0.0,0.0,ADNI2
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
...,...,...,...,...,...
24726,041_S_1423,2.0,1.0,1.0,ADNI1
24735,082_S_1377,2.0,2.0,2.0,ADNI1
24827,041_S_1435,2.0,2.0,2.0,ADNI1
24964,128_S_1430,2.0,2.0,2.0,ADNI1


In [35]:
m4[m4["GroupN"] != m4["PHC_Diagnosis"]]

,Subject,GroupN,Group,PHC_Diagnosis,Phase
0,011_S_0002,1.0,0.0,0.0,ADNI1
9,011_S_0002,1.0,0.0,0.0,ADNIGO
10,011_S_0002,1.0,0.0,0.0,ADNI2
77,011_S_0008,1.0,0.0,0.0,ADNI1
102,011_S_0008,1.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
24391,022_S_1394,2.0,1.0,1.0,ADNI1
24453,041_S_1412,2.0,1.0,1.0,ADNI1
24618,041_S_1425,2.0,1.0,1.0,ADNI1
24655,127_S_1427,2.0,1.0,1.0,ADNI1


In [36]:
m2[["Subject", "GroupN", "Group", "PHC_Diagnosis", "Phase"]]

,Subject,GroupN,Group,PHC_Diagnosis,Phase
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
119,022_S_0007,2.0,2.0,2.0,ADNI1
197,023_S_0031,0.0,0.0,0.0,ADNI1
233,023_S_0031,0.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
24617,127_S_1419,1.0,1.0,1.0,ADNI2
24735,082_S_1377,2.0,2.0,2.0,ADNI1
24827,041_S_1435,2.0,2.0,2.0,ADNI1
24964,128_S_1430,2.0,2.0,2.0,ADNI1


In [88]:
m5 = pd.concat([m1,m3,m4])
i = m5[m5["Group"] == m5["GroupN"]]
i = i[i["Group"] == i["PHC_Diagnosis"]]

In [89]:
i = i.drop_duplicates()

In [90]:
i

,Subject,GroupN,Group,PHC_Diagnosis,Phase
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
119,022_S_0007,2.0,2.0,2.0,ADNI1
197,023_S_0031,0.0,0.0,0.0,ADNI1
233,023_S_0031,0.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
54839,013_S_7118,0.0,0.0,0.0,ADNI3
54840,153_S_7113,0.0,0.0,0.0,ADNI3
54841,035_S_7120,2.0,2.0,2.0,ADNI3
54844,082_S_7117,1.0,1.0,1.0,ADNI3


In [91]:
i

,Subject,GroupN,Group,PHC_Diagnosis,Phase
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
119,022_S_0007,2.0,2.0,2.0,ADNI1
197,023_S_0031,0.0,0.0,0.0,ADNI1
233,023_S_0031,0.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
54839,013_S_7118,0.0,0.0,0.0,ADNI3
54840,153_S_7113,0.0,0.0,0.0,ADNI3
54841,035_S_7120,2.0,2.0,2.0,ADNI3
54844,082_S_7117,1.0,1.0,1.0,ADNI3


In [92]:
len(i['Subject'].unique())

2738

In [40]:
i[["Subject", "Group", "Phase"]].to_csv("ground_truth.csv")

In [41]:
m.update(m5[~m5.index.duplicated(keep='first')])

In [42]:
indexes = m.index

In [43]:
#if none of the three diagnosis agree, then we set the value to -1
m["GROUP"] = -1

In [95]:
print("m1:", m1[m1["GroupN"] == -2])
print("m3:", m3[m3["GroupN"] == -2])
print("m4:", m4[m4["GroupN"] == -2])

m1:            Subject  GroupN  Group  PHC_Diagnosis  Phase
28177   032_S_4429    -2.0   -2.0           -2.0  ADNI3
31514   011_S_4278    -2.0   -2.0           -2.0  ADNI3
47870   130_S_5175    -2.0   -2.0           -2.0  ADNI3
53166   011_S_6618    -2.0   -2.0           -2.0  ADNI3
53719   012_S_6760    -2.0   -2.0           -2.0  ADNI3
...            ...     ...    ...            ...    ...
55126  114_S_10321    -2.0   -2.0           -2.0    NaN
55127  082_S_10219    -2.0   -2.0           -2.0    NaN
55128  033_S_10326    -2.0   -2.0           -2.0    NaN
55129  011_S_10294    -2.0   -2.0           -2.0    NaN
55130  014_S_10335    -2.0   -2.0           -2.0    NaN

[281 rows x 5 columns]
m3: Empty DataFrame
Columns: [Subject, GroupN, Group, PHC_Diagnosis, Phase]
Index: []
m4:           Subject  GroupN  Group  PHC_Diagnosis   Phase
279    011_S_0021    -2.0    0.0            0.0   ADNI1
304    011_S_0021    -2.0    0.0            0.0  ADNIGO
305    011_S_0021    -2.0    0.0          

In [44]:
for i in indexes:
    row = m.loc[i]
    if (row["GroupN"] == row["Group"]):
        val = row["GroupN"]
        
        m.loc[i, "GROUP"] = val
    elif (row["GroupN"] == row["PHC_Diagnosis"]):
        val = row["GroupN"]
        m.loc[i, "GROUP"] = val
        
    elif (row["Group"] == row["PHC_Diagnosis"]):
        val = row["Group"]
        m.loc[i, "GROUP"] = val
        

In [45]:
m5 = m5[~m5.index.duplicated(keep='first')]
m5

,Subject,GroupN,Group,PHC_Diagnosis,Phase
36,011_S_0003,2.0,2.0,2.0,ADNI1
52,011_S_0005,0.0,0.0,0.0,ADNI1
119,022_S_0007,2.0,2.0,2.0,ADNI1
197,023_S_0031,0.0,0.0,0.0,ADNI1
233,023_S_0031,0.0,0.0,0.0,ADNIGO
...,...,...,...,...,...
24391,022_S_1394,2.0,1.0,1.0,ADNI1
24453,041_S_1412,2.0,1.0,1.0,ADNI1
24618,041_S_1425,2.0,1.0,1.0,ADNI1
24655,127_S_1427,2.0,1.0,1.0,ADNI1


In [46]:
m[m["GROUP"] != -1]

,Subject,GroupN,Group,PHC_Diagnosis,Phase,DX,GROUP
0,011_S_0002,1.0,0.0,0.0,ADNI1,NaN,0
9,011_S_0002,1.0,0.0,0.0,ADNIGO,NaN,0
10,011_S_0002,1.0,0.0,0.0,ADNI2,NaN,0
11,011_S_0002,1.0,0.0,1.0,ADNI2,NaN,1
35,011_S_0002,1.0,0.0,1.0,ADNI3,NaN,1
...,...,...,...,...,...,...,...
55126,114_S_10321,-2.0,-2.0,-2.0,NaN,NaN,-2
55127,082_S_10219,-2.0,-2.0,-2.0,NaN,NaN,-2
55128,033_S_10326,-2.0,-2.0,-2.0,NaN,NaN,-2
55129,011_S_10294,-2.0,-2.0,-2.0,NaN,NaN,-2


In [47]:
m[["Subject", "GroupN", "Group", "PHC_Diagnosis", "GROUP", "Phase"]].to_csv("diagnosis_full.csv")